In [1]:
import os

# check the current working dir
%pwd

'g:\\ML_DL_Projects\\MLOPS_Indian_Flight_Price_Prediction\\research'

In [2]:
# changing the dir to main repo
os.chdir("../")

%pwd

'g:\\ML_DL_Projects\\MLOPS_Indian_Flight_Price_Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

# step 1 in workflow for Data ingestion
@dataclass
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [ ]:
# step 2 and 3 not needed now
# step 4 later on
# step 5 in workflow for data ingestion

from src.indian_flight_price_prediction.constants import *
from src.indian_flight_price_prediction.utils.common import read_yaml, create_directories


In [4]:
class ConfigurationMaanger:
    def __init__(self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH,
                schema_file_path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories(self.config.artifacts_root)

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories(config.root_dir)

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
from src.indian_flight_price_prediction.logger import logger
import zipfile

In [ ]:
# compnent - Data Ingestion 

class DataIngestion:
    def __init__(self, config : DataIngestionConfig ):
        self.config = config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists.")

    # extract zip file
    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r' ) as zip_ref:
            zip_ref.extractall(unzip_path)